In [26]:
import torch
import torch.nn as nn
from transformers import (
    CLIPModel, CLIPProcessor, GPT2LMHeadModel, GPT2Tokenizer,
    AutoModelForCausalLM, AutoTokenizer
)

c:\Users\MTSS\VirtualEnvs\dev-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

c:\Users\MTSS\VirtualEnvs\dev-env\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
tokens = tokenizer('<|endoftext|>', return_tensors='pt')

print(tokens)


{'input_ids': tensor([[50256]]), 'attention_mask': tensor([[1]])}


In [36]:
bos_token_id = tokens['input_ids'][0][0]

bos_token_id

tensor(50256)

In [37]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

c:\Users\MTSS\VirtualEnvs\dev-env\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\MTSS\VirtualEnvs\dev-env\lib\site-packages\transformers\modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the 

In [38]:
gpt2_token_emb_layer = model.transformer.wte

In [39]:
# start_token_embedding = embedding_layer(torch.IntTensor([1]))
bos_emb = gpt2_token_emb_layer(bos_token_id)


In [40]:
bos_emb.shape

torch.Size([768])

In [6]:
import pickle

p = pickle.load(open('dataset.pkl', 'rb'))

p[0][1].shape


In [1]:
from todo import CaptioningModel

c:\Users\MTSS\VirtualEnvs\dev-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
CaptioningModel(
    clip_model='openai/clip-vit-base-patch32',
    text_model='gpt2',
    ep_len=4,
    num_layers=1,
    n_heads=1,
    forward_expansion=
)